# Current Covid API Data

In [33]:
import pandas as pd
import requests

In [7]:
df_curr = pd.read_csv('https://api.covidtracking.com/v1/states/current.csv')

In [23]:
cols = ['state', 'positive']

In [26]:
df_curr[cols].shape

(56, 2)

In [28]:
df_curr[df['positive'] == 0]

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
3,20200824,AS,0,1514,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c878baea61754904c3c233d78e523723e2f4a8e1,0,0,0,0,0,NaN


# Historical Covid API Data

In [30]:
df_hist = pd.read_csv('https://api.covidtracking.com/v1/states/daily.csv')

In [31]:
df_hist.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200824,AK,5628.0,324875.0,NaN,43.0,NaN,NaN,NaN,8.0,...,330503,0,0,54865e3eaef9af79f98d4ca570197186492760a0,0,0,0,0,0,NaN
1,20200824,AL,116710.0,806609.0,NaN,1149.0,13793.0,NaN,1391.0,NaN,...,923319,11,291,2fe1d500d143b2a7f02117934c2c53d0b2648560,0,0,0,0,0,NaN
2,20200824,AR,56894.0,616810.0,NaN,466.0,3962.0,NaN,NaN,108.0,...,673704,9,42,51dc585c49ed43ee35e01760e30b6b7aa792afe6,0,0,0,0,0,NaN
3,20200824,AS,0.0,1514.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1514,0,0,c878baea61754904c3c233d78e523723e2f4a8e1,0,0,0,0,0,NaN
4,20200824,AZ,198414.0,958808.0,NaN,908.0,21219.0,345.0,NaN,176.0,...,1157222,0,-22,a3a22a4967b6ca06481f5aa38b1c59235db23f87,0,0,0,0,0,NaN


In [ ]:
https://api.covidtracking.com/v1/states/daily.csv